<a href="https://colab.research.google.com/github/MukulRaj-704/Spam-Detector/blob/main/Spam_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
paths = '/content/drive/My Drive/Colab-files/enron emails.csv'

In [5]:
df=pd.read_csv(paths, encoding='latin1')

In [6]:
df.shape

(33784, 3)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33784 entries, 0 to 33783
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Message     33781 non-null  object
 1   Category    33701 non-null  object
 2   Unnamed: 2  3 non-null      object
dtypes: object(3)
memory usage: 791.9+ KB


Here we calculate the total no of missing values in our Message column od dataset

In [8]:
# Missing_mask = df['Message'].isnull()
# print(Missing_mask)
# Missing_mask.sum()
# print(Missing_mask.sum())


# Alternate to do the same thing above
total_missing_in_column = df['Message'].isnull().sum()
print(total_missing_in_column)

3


Now we drop these missing Vlaues in Message column

In [9]:
drop_df = df.dropna(subset=['Message'])
print(drop_df)

                                                 Message Category Unnamed: 2
0               Subject: christmas tree farm pictures\r       ham        NaN
1                Subject: re : rankings\r thank you .\r       ham        NaN
2      Subject: leadership development pilot\r sally ...      ham        NaN
3      Subject: key dates and impact of upcoming sap ...      ham        NaN
4      Subject: key hr issues going forward a ) year ...      ham        NaN
...                                                  ...      ...        ...
33779  Subject: do you wish for a more active sex lif...     spam        NaN
33780  Subject: bloow in 5 - 10 times the time\r lear...     spam        NaN
33781  Subject: why didnt you say hi\r there is this ...     spam        NaN
33782  Subject: dear sir , i am interested in it\r hi...     spam        NaN
33783  Subject: \r html\r body\r b\r pwhy not take a ...     spam        NaN

[33781 rows x 3 columns]


so we have 33781 rows lest and 3 rows are deleted

In [10]:
drop_df.describe()
drop_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33781 entries, 0 to 33783
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Message     33781 non-null  object
 1   Category    33698 non-null  object
 2   Unnamed: 2  0 non-null      object
dtypes: object(3)
memory usage: 1.0+ MB


Preprocessing Data

Convert All Email into LowerCase

In [11]:
# df = pd.read_csv("/content/drive/My Drive/Colab-files/enron emails.csv", encoding='latin1')
drop_df['Message'] = drop_df['Message'].str.lower()
drop_df.to_csv("updated_file.csv", index=False)

/tmp/ipython-input-11-235639736.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drop_df['Message'] = drop_df['Message'].str.lower()


In [12]:
drop_df.head()

,Message,Category,Unnamed: 2
0,subject: christmas tree farm pictures\r,ham,NaN
1,subject: re : rankings\r thank you .\r,ham,NaN
2,subject: leadership development pilot\r sally ...,ham,NaN
3,subject: key dates and impact of upcoming sap ...,ham,NaN
4,subject: key hr issues going forward a ) year ...,ham,NaN


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=3000)  # optional: limit vocabulary size
X_tfidf = tfidf_vectorizer.fit_transform(drop_df['Message'])

In [16]:
print(f"Shape of X_tfidf (documents, features): {X_tfidf.shape}")

Shape of X_tfidf (documents, features): (33781, 3000)


In [18]:
feature_names = tfidf_vectorizer.get_feature_names_out()
print("\n--- First 20 TF-IDF Feature Names (Words/N-grams in Vocabulary) ---")
print(feature_names[:20])
# ...
#print(df_tfidf_output)


--- First 20 TF-IDF Feature Names (Words/N-grams in Vocabulary) ---
['00' '000' '0000' '01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '100'
 '1000' '101' '105' '11' '110' '12']


In [22]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# --- 1. Define the text preprocessing function ---
def preprocess_text(text):
    text = str(text) # Ensure it's a string
    text = text.lower() # Convert to lowercase

    # Remove URLs (e.g., http://..., www. etc.)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove email addresses
    text = re.sub(r'\S*@\S*\s?', '', text)
    # Remove purely numeric tokens (e.g., '12345', '2001')
    # \b ensures word boundaries, so 'abc123def' is not affected, only '123' as a standalone word
    text = re.sub(r'\b\d+\b', '', text)

    # Remove punctuation and special characters, replace with a space
    # [^\w\s] matches any character that is NOT a word character (alphanumeric + underscore) or whitespace
    text = re.sub(r'[^\w\s]', ' ', text)

    # Remove extra whitespaces (e.g., multiple spaces, newlines, tabs) and strip leading/trailing spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# --- 2. Define the main function to process the CSV and apply TF-IDF ---
def process_and_tfidf(input_csv_file, message_column):
    df = None
    possible_encodings = ['utf-8', 'latin1', 'cp1252', 'ISO-8859-1']

    # Try to load the CSV with different encodings
    for encoding_try in possible_encodings:
        try:
            print(f"Attempting to read '{input_csv_file}' with encoding: '{encoding_try}'...")
            # Using header=None and names based on previous discussions about your file structure
            df = pd.read_csv(input_csv_file, encoding=encoding_try, header=None, names=['EmailContent', 'Label', 'ExtraColumn'])
            print(f"Successfully loaded '{input_csv_file}' with encoding '{encoding_try}'.")
            break
        except UnicodeDecodeError:
            print(f"Failed to decode with '{encoding_try}'. Trying next encoding...")
        except FileNotFoundError:
            print(f"Error: The file '{input_csv_file}' was not found. Please check the path and filename.")
            return
        except Exception as e:
            print(f"An unexpected error occurred during file reading: {e}")
            return

    if df is None:
        print("Could not read the file with any of the common encodings. Please check the file's encoding manually.")
        return

    print(f"\n--- Original DataFrame loaded successfully ---")
    print(f"Shape of loaded DataFrame (rows, columns): {df.shape}")
    print(f"First 5 rows of loaded DataFrame:")
    print(df.head())
    print(f"Available columns are: {df.columns.tolist()}")

    # --- 3. Drop missing values in the message column and preprocess text ---
    if message_column not in df.columns:
        print(f"Error: Column '{message_column}' not found. Please ensure it matches one of: {df.columns.tolist()}")
        return

    # Drop rows where the message column has NaN values
    print(f"\nDropping rows with missing values in '{message_column}' column...")
    df_cleaned = df.dropna(subset=[message_column])
    print(f"Shape after dropping NaNs: {df_cleaned.shape}")

    # Apply the preprocessing function to the cleaned message column
    print(f"Applying text preprocessing to '{message_column}' to create 'Message_processed'...")
    df_cleaned['Message_processed'] = df_cleaned[message_column].apply(preprocess_text)

    print("\n--- Processed Messages Head (important to check this output) ---")
    # This print will show the raw vs processed message content
    print(df_cleaned[[message_column, 'Message_processed']].head(10)) # Print 10 rows to see more context

    # --- 4. Initialize and apply TfidfVectorizer ---
    print(f"\nInitializing TfidfVectorizer with max_features=3000 and custom token_pattern...")
    tfidf_vectorizer = TfidfVectorizer(
      #  max_features=3000,
        lowercase=True,         # Will be applied internally, but our text is already lowercase
        stop_words='english',   # Remove common English stop words
        min_df=2,               # Ignore words appearing in less than 2 documents
        ngram_range=(1, 2),     # Include unigrams and bigrams
        token_pattern=r'[a-zA-Z]{2,}' # <--- CRUCIAL: Only tokenize words with at least 2 alphabetic characters
    )

    print("Fitting and transforming data with TfidfVectorizer on 'Message_processed'...")
    X_tfidf = tfidf_vectorizer.fit_transform(df_cleaned['Message_processed']) # <--- Use 'Message_processed' here

    # --- 5. See the output of X_tfidf ---
    print(f"\n--- TF-IDF Vectorization Complete ---")
    print(f"Shape of X_tfidf (documents, features): {X_tfidf.shape}")

    feature_names = tfidf_vectorizer.get_feature_names_out()
    print("\n--- First 20 TF-IDF Feature Names (Words/N-grams in Vocabulary) ---")
    print(feature_names[:20]) # Print first 20 to check for numerical tokens

    # Convert a small portion of the sparse matrix to dense for viewing
    num_rows_to_display = min(5, X_tfidf.shape[0]) # Display up to 5 rows, or fewer if less data

    if num_rows_to_display > 0:
        X_tfidf_dense_head = X_tfidf[:num_rows_to_display].toarray()
        df_tfidf_output = pd.DataFrame(X_tfidf_dense_head, columns=feature_names)
        print(f"\n--- First {num_rows_to_display} Rows of TF-IDF Output (Document-Term Matrix) ---")
        print(df_tfidf_output)
    else:
        print("\nNo data to display in TF-IDF matrix (possibly all rows dropped or empty after processing).")

# --- Main execution block ---
if __name__ == "__main__":
    # --- IMPORTANT: Ensure this matches your file name in the environment ---
    INPUT_CSV = "/content/drive/My Drive/Colab-files/enron emails.csv" # The file you uploaded
    # This must match the name assigned in `names` parameter during read (e.g., 'EmailContent')
    EMAIL_CONTENT_COLUMN_NAME = "EmailContent"

    process_and_tfidf(INPUT_CSV, EMAIL_CONTENT_COLUMN_NAME)

Attempting to read '/content/drive/My Drive/Colab-files/enron emails.csv' with encoding: 'utf-8'...
Failed to decode with 'utf-8'. Trying next encoding...
Attempting to read '/content/drive/My Drive/Colab-files/enron emails.csv' with encoding: 'latin1'...
Successfully loaded '/content/drive/My Drive/Colab-files/enron emails.csv' with encoding 'latin1'.

--- Original DataFrame loaded successfully ---
Shape of loaded DataFrame (rows, columns): (33785, 3)
First 5 rows of loaded DataFrame:
                                        EmailContent     Label ExtraColumn
0                                            Message  Category         NaN
1           Subject: christmas tree farm pictures\r        ham         NaN
2            Subject: re : rankings\r thank you .\r        ham         NaN
3  Subject: leadership development pilot\r sally ...       ham         NaN
4  Subject: key dates and impact of upcoming sap ...       ham         NaN
Available columns are: ['EmailContent', 'Label', 'ExtraColu

/tmp/ipython-input-22-1777981218.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Message_processed'] = df_cleaned[message_column].apply(preprocess_text)



--- Processed Messages Head (important to check this output) ---
                                        EmailContent  \
0                                            Message   
1           Subject: christmas tree farm pictures\r    
2            Subject: re : rankings\r thank you .\r    
3  Subject: leadership development pilot\r sally ...   
4  Subject: key dates and impact of upcoming sap ...   
5  Subject: key hr issues going forward a ) year ...   
6  Subject: re : quasi\r good morning ,\r i ' d l...   
7  Subject: vastar resources , inc .\r gary , pro...   
8  Subject: congrats !\r contratulations on the e...   
9  Subject: re : visit to enron\r vince ,\r dec ....   

                                   Message_processed  
0                                            message  
1               subject christmas tree farm pictures  
2                      subject re rankings thank you  
3  subject leadership development pilot sally wha...  
4  subject key dates and impact of upcomin

In [25]:
print(feature_names[:200])

['00' '000' '0000' '01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '100'
 '1000' '101' '105' '11' '110' '12' '120' '125' '13' '14' '140' '1400'
 '15' '150' '16' '161' '1618' '17' '18' '19' '1933' '1934' '1995' '1997'
 '1998' '1999' '20' '200' '2000' '2001' '2002' '2003' '2004' '2005' '21'
 '212' '215' '22' '229' '23' '24' '25' '250' '26' '27' '28' '281' '29'
 '30' '300' '31' '32' '33' '34' '345' '35' '36' '37' '38' '39' '40' '400'
 '41' '42' '43' '44' '45' '46' '464' '47' '48' '49' '50' '500' '503' '51'
 '512' '52' '53' '54' '55' '56' '57' '58' '59' '60' '600' '61' '62' '63'
 '64' '646' '65' '650' '66' '67' '68' '69' '70' '700' '713' '72' '75'
 '750' '76' '77002' '78' '79' '80' '800' '84' '85' '853' '87' '8859' '888'
 '89' '90' '92' '95' '96' '97' '971' '98' '99' 'aa' 'ability' 'able'
 'about' 'above' 'absolutely' 'abuse' 'ac' 'accept' 'acceptance'
 'accepted' 'access' 'according' 'account' 'accounting' 'accounts'
 'accuracy' 'achieve' 'acquire' 'acquired' 'acquisition' 'acquisitions

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Labels (0 = ham, 1 = spam)
y = df_cleaned_category['Category'].map({'ham': 0, 'spam': 1})

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Train Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Evaluate model
y_pred = nb_model.predict(X_test)
print("📊 Model Evaluation on Test Set:")
print(classification_report(y_test, y_pred))

# -----------------------------------
# 📥 Predict on Custom Email
# -----------------------------------

# Custom email to classify
email = ["Dear Mukul Raj Paawan, We hope you're doing great! Congratulations on your next leap towards a maximized career with Flipkart! You have successfully completed your assessment..."]

# Vectorize the email using same tfidf_vectorizer
email_vectorized = tfidf_vectorizer.transform(email)

# Predict
prediction = nb_model.predict(email_vectorized)
confidence = nb_model.predict_proba(email_vectorized)

# Output
print("\n📩 Custom Email Prediction:")
print("Prediction:", "Spam 🚨" if prediction[0] == 1 else "Ham ✅")
print("Confidence (Ham, Spam):", confidence[0])


📊 Model Evaluation on Test Set:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      3300
           1       0.97      0.99      0.98      3440

    accuracy                           0.98      6740
   macro avg       0.98      0.98      0.98      6740
weighted avg       0.98      0.98      0.98      6740


📩 Custom Email Prediction:
Prediction: Ham ✅
Confidence (Ham, Spam): [0.5814499 0.4185501]


In [59]:
# List of custom emails
custom_emails = [
    "Dear Mukul Raj Paawan, We hope you're doing great! Congratulations on your next leap towards a maximized career with Flipkart! You have successfully completed your assessment...",
    "Get FREE access to our exclusive offer — click now and claim your reward!",
    "Hi Mukul, your resume was shortlisted. Please schedule your interview at your convenience.",
    "Warning: Your account will be suspended unless you verify your identity now. Click here to confirm."
]

# Loop through each email and make prediction
for i, email in enumerate(custom_emails, 1):
    email_vectorized = tfidf_vectorizer.transform([email])
    prediction = nb_model.predict(email_vectorized)
    confidence = nb_model.predict_proba(email_vectorized)

    print(f"\n📧 Email {i}:")
    print(email)
    print("Prediction:", "Spam 🚨" if prediction[0] == 1 else "Ham ✅")
    print("Confidence (Ham, Spam):", confidence[0])



📧 Email 1:
Dear Mukul Raj Paawan, We hope you're doing great! Congratulations on your next leap towards a maximized career with Flipkart! You have successfully completed your assessment...
Prediction: Ham ✅
Confidence (Ham, Spam): [0.5814499 0.4185501]

📧 Email 2:
Get FREE access to our exclusive offer — click now and claim your reward!
Prediction: Spam 🚨
Confidence (Ham, Spam): [0.01564332 0.98435668]

📧 Email 3:
Hi Mukul, your resume was shortlisted. Please schedule your interview at your convenience.
Prediction: Ham ✅
Confidence (Ham, Spam): [0.95454375 0.04545625]

📧 Email 4:
Prediction: Spam 🚨
Confidence (Ham, Spam): [0.03414148 0.96585852]
